<a href="https://colab.research.google.com/github/jdariasl/ML_2020/blob/master/Labs/lab6/lab6_parte1.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Recuerda que una vez abierto, Da clic en "Copiar en Drive", de lo contrario no podras almacenar tu progreso**

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

In [ ]:
#configuración del laboratorio
# Ejecuta esta celda!
%load_ext autoreload
%autoreload 2
#for local 
#import sys ; sys.path.append('../commons/utils/')
!wget https://raw.githubusercontent.com/jdariasl/ML_2020/master/Labs/commons/utils/general.py -O general.py --no-cache
from general import configure_lab6
configure_lab6()
from lab6 import *
GRADER, x,y = part_1()

# Laboratorio 6 - Parte 1: Reducción de dimensión y Selección de características

Este ejercicio tiene como objetivo implementar varias técnicas de selección de características y usar SVM para resolver un problema de clasificación multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography

Analice la base de datos, sus características, su variable de salida y el contexto del problema.

In [ ]:
print('Dimensiones de la base de datos de entrenamiento. dim de x: ' + str(np.shape(x)) + '\tdim de y: ' + str(np.shape(y)))

**observación para las librerias sklearn **

Llamar explicitamente los parametros de las librerias de sklearn (e.j. si se quiere usar el parametro `kernel` del `SVC`, se debe llamar `SVC(kernel='rbf'`)

## Ejercicio 1: Entrenamiento sin selección de características

En nuestro primer ejercicio debemos completar la función para entrenar una SVM para resolver un problema de clasificación. Debemos completar siguiendo las recomendaciones:

1. Mantener los parámetros sugeridos del SVC. 
2. Asignar el parametro de Kfold a los splits
3. Usar la exactitud como medida de error del modulo [metrics de sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)
4. Esta función la vamos a usar como base para comparar nuestros metodos de selección de características.

In [ ]:
#ejercicio de código
def entrenamiento_sin_seleccion_caracteristicas(splits, X, Y):
    """
    Función que ejecuta el entrenamiento del modelo sin una selección particular
    de las características

      Parámetros:splits : numero de particiones  a realizar
      Retorna:
      1. El modelo entreando
      2. El vector de errores
      3. El Intervalo de confianza
      4. El tiempo de procesamiento
    """
    #Implemetamos la metodología de validación
    Errores = np.ones(splits)
    Score = np.ones(splits)
    times = np.ones(splits)
    j = 0
    kf = KFold(...)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        #Creamos el clasificador SVM.
        clf = SVC(kernel="linear", C=1)
        #Aquí se entran y se valida el modelo sin hacer selección de características
        ######
        # Entrenamiento el modelo.
        #Para calcular el costo computacional
        tiempo_i = time.time()
        clf.fit(X_train,y_train)
        # Validación del modelo
        Errores[j] = ...(y_true=..., y_pred=clf.predict(...))
        times[j] = time.time()-tiempo_i
        j+=1

    return clf, np.mean(Errores), np.std(Errores), np.mean(times)

In [ ]:
## la funcion que prueba tu implementacion
GRADER.run_test("ejercicio1", entrenamiento_sin_seleccion_caracteristicas)

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Cuáles son los parámetros establecidos para el modelo?
respuesta_1 = '' #@param {type:"string"}

## Ejercicio 2: Entrenamiento con selección de características

La siguiente función "wrapper" nos permite hacer una selección de características utilizando la [librería recursive feature elimination de Sci-kit Learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html).

Esta libreria es un metodo de seleccion carcterisitcas wrapper, que usa los coeficientes derivados de  un estimador entrenado para estimar que caracteristicas tienen mayor poder predictivo.

Para completar debemos tener en cuenta lo siguiente:

1. Para el número de caractersiticas usar el parametro feature_numbers
2. Establecer el paso = 1 para ir eliminando las caracteristicas
3. Asumir que el estimador se crea externamente de la función
4. Entender los campos del RFE disponibles despues de entrenarlo, para obtener:
    1. La mascara para saber que características fueron seleccionadas
    2. El ranking de las caracteristicas

In [ ]:
#ejercicio de código
def recursive_feature_elimination_wrapper(estimator, feature_numbers, X,Y):
    """
    Esta función es un envoltorio del objeto RFE de sklearn

    Parámetros:
    estimator(sklearn.svm.SVC), El estimador SVM
    feature_numbers(int), El número de características a considerar
    X (numpy.array), El arreglo numpy de características
    Y (numpy.array), El vector de etiquetas

    Retorna:
    El modelo entrenado ()
    La máscara de características seleccionada, array [longitud de caracterisitcas de X]
    El rankeo de características, array [longitud de caracterisitcas de X]
    El objeto RFE entrenado sobre el set reducido de características
    El tiempo de ejecución
    """
    rfe = RFE(estimator=estimator, ...)
    tiempo_i = time.time()
    rfe.fit(...)
    time_o = time.time()-tiempo_i
    feature_mask = rfe...
    features_rank = rfe...
    estimator = rfe...

    return rfe, feature_mask, features_rank, estimator, time_o

In [ ]:
## la funcion que prueba tu implementacion
GRADER.run_test("ejercicio2", recursive_feature_elimination_wrapper)

In [ ]:
#@title Preguntas Abierta
#@markdown ¿Que diferencia tiene el metodo implementado con un metodo de filtro? explicar
respuesta_2 = '' #@param {type:"string"}

## Ejercicio 3:  Comparación de los resultados del modelo

Ahora en la siguiente función, vamos a usar la función planteada para realizar experimentos con la selección de características. Para ello:
1. Utilizar una metodología cross-validation.
2. Usar como parametros para los experimentos el número de características y el número de splits/particiones
3. Usar la función `recursive_feature_elimination_wrapper` para realizar la selección de características.
3. Vamos a retornar un DataFrame con las siguientes columnas:
    - CON_SEL (indicando si se uso selección de caracteristicas)
    - NUM_VAR (número de selección de caracteristicas)
    - NUM_SPLITS  (número de particiones realizadas)
    - ERROR_VALIDACION
    - IC_STD_VALIDACION
    - T_EJECUCION: tiempo de ejecucción
4. En las primeras filas del dataframe vamos a incluir la evaluación del modelo SVM sin selección de características (usando la función creada en el primer ejercicio). 

In [ ]:
#ejercicio de código
def experimentar(n_feats, n_sets, X, Y):
    """
    Esta función realiza la comparación del desempeño de RFE utilizando diferente 
    número de feats y particionando el conjunto de datos en diferente número de 
    subconjuntos

    Parámetros:
    X (numpy.array), El arreglo numpy de características
    Y (numpy.array), El vector de etiquetas
    n_feats, Vector de números enteros que indica el número de características
              que debe utilizar el modelo
    n_sets, Vector de números enteros que indica el número de particiones

    Retorna:  
    - DataFrame con las columnas: CON_SEL, NUM_VAR, NUM_SPLITS, ERROR_VALIDACION, IC_STD_VALIDACION, 
    y T_EJECUCION. 

    """
    df = pd.DataFrame()
    idx = 0
    for split_number in n_sets: 
    #Sin selección de características
        # se ignorar las otras salidas
        _,err,ic,t_ex = ...(split_number, X,Y)  
        df.loc[idx,'CON_SEL'] = 'NO'
        df.loc[idx,'NUM_VAR'] = X.shape[1]
        df.loc[idx,'NUM_SPLITS'] = ...
        df.loc[idx,'ERROR_VALIDACION'] = ...
        df.loc[idx,'IC_STD_VALIDACION'] = ...
        df.loc[idx,'T_EJECUCION'] = ...
        idx+=1
    print("termina experimentos sin selección")
    #Con selección de características
    for f in n_feats:
        for split_number in n_sets:
            #Implemetamos la metodología de validación 
            Errores = np.ones(split_number)
            Score = np.ones(split_number)
            times = np.ones(split_number)
            kf = ...(n_splits=split_number)
            j = 0
            for train_index, test_index in kf.split(X):
                
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = Y[train_index], Y[test_index]
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
                
                svc =  SVC(kernel="linear", C=1)
                
                # se ignorar las otras salidas
                rfe, _, _, _, t = ...(estimator=svc,feature_numbers=..., X=...,Y=...)
            
                Errores[j]=accuracy_score(y_true = y_test,  y_pred=rfe...)
                times[j] = t
                j+=1

            df.loc[idx,'CON_SEL'] = 'SI'
            df.loc[idx,'NUM_VAR'] = f
            df.loc[idx,'NUM_SPLITS'] = split_number
            df.loc[idx,'ERROR_VALIDACION'] = np.mean(Errores)
            df.loc[idx, 'IC_STD_VALIDACION'] = np.std(Errores)
            df.loc[idx, 'T_EJECUCION'] = np.mean(times)
            idx+=1
    return df


In [ ]:
## la funcion que prueba tu implementacion
GRADER.run_test("ejercicio3", experimentar)

In [ ]:
#@title Preguntas Abierta
#@markdown ¿Con qué modelo base se está resolviendo la selección de características?
respuesta_3 = '' #@param {type:"string"}

Ejecuta la celda de codigo para realizar los experimentos

In [ ]:
dfr = experimentar(n_feats = [3, 5, 10,15,20], n_sets = [5, 10], X= x, Y=y)

En la siguiente celda vamos a ordenar los resultados con base a el error de validación y el tiempo de ejecucción.

In [ ]:
dfr.sort_values(['ERROR_VALIDACION','T_EJECUCION'], ascending=[False, True])

Y ahora vamos a ver como se relaciona el tiempo de ejecución con los splits, la selección de caracteristicas y le rendimiento del modelo

In [ ]:
import seaborn as sns
d_toplot = pd.melt(dfr,id_vars=['CON_SEL', 'NUM_VAR', 'NUM_SPLITS'], value_vars=['ERROR_VALIDACION', 'T_EJECUCION'])
sns.relplot(data = d_toplot, x = 'NUM_VAR', y = 'value', hue = 'CON_SEL', style = 'NUM_SPLITS', col = 'variable', kind='scatter', facet_kws = {'sharey' : False}, aspect=1.2,s=150)

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Que relación observa entre tiempo de ejecución, el desempeño del modelo y el número de caracteristicas? Explicar con base a los resultados
respuesta_4 = '' #@param {type:"string"}

Ahora use el número de características que lograron el mejor desempeño para entrenar nuevamente un modelo y saber que características tienen el mejor poder predictivo.

In [ ]:
svc =  SVC(kernel="linear", C=1)
rfe, feature_mask, _, _, _ = recursive_feature_elimination_wrapper(svc, ..., x,y)
print("esta es la mascara (deberia ser solo valores True y False) \n", feature_mask)

In [ ]:
#@title Pregunta Abierta
#@markdown Utilizando los resultados obtenidos ¿cuál sería su sugerencia de apoyo diagnóstico para un médico que esté trabajando en un caso enmarcado en esta base de datos?
respuesta_5 = '' #@param {type:"string"}

In [ ]:
GRADER.check_tests()

In [ ]:
#@title Integrantes
codigo_integrante_1 ='' #@param {type:"string"}
codigo_integrante_2 = ''  #@param {type:"string"}

----
esta linea de codigo va fallar, es de uso exclusivo de los profesores


In [ ]:
GRADER.grade()